In [1]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

c:\Users\tomokinakamura\anaconda3\envs\robots_town\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset

# CIFAR-10データセットの基本変換
transform_base = transforms.Compose([
    transforms.ToTensor(),  # Tensorに変換
])
train_cifar10_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_base)
test_cifar10_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_base)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
from torchvision import models
from torch.nn import init
N = 5
agents_models = {}
for i in range(N):
    # ResNet18モデルを定義してエージェントに割り当て
    model = models.resnet18(pretrained=False)  # pretrainedをFalseに設定
    model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 10)  # 出力層を10クラスに変更
    for param in model.parameters():
        init.constant_(param, 0)

    agents_models[i] = model

c:\Users\tomokinakamura\anaconda3\envs\robots_town\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tomokinakamura\anaconda3\envs\robots_town\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
from torch.utils.data import random_split
from torchvision import datasets
from torch.utils.data.dataset import Subset

train_dataset_size = len(train_cifar10_dataset)
per_agent_size = train_dataset_size // N
sizes = [per_agent_size] * N
sizes[-1] += train_dataset_size - sum(sizes)
train_agents_datasets = random_split(train_cifar10_dataset, sizes)
test_dataset_size = len(test_cifar10_dataset)
test_per_agent_size = test_dataset_size // N
test_sizes = [test_per_agent_size] * N
test_sizes[-1] += test_dataset_size - sum(test_sizes)
test_agents_datasets = random_split(test_cifar10_dataset, test_sizes)

In [5]:
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_function = nn.CrossEntropyLoss()

In [6]:
param_shapes = [param.shape for param in agents_models[0].parameters()]
param_elements = [param.numel() for param in agents_models[0].parameters()]

def unflatten_parameters(flat_params, param_shapes, param_elements):
    params = []
    offset = 0
    for shape, elements in zip(param_shapes, param_elements):
        param = flat_params[offset:offset + elements].view(shape)
        params.append(param)
        offset += elements
    return params

def assign_params_to_model(model, params):
    with torch.no_grad():  # 勾配計算を無効化
        for param, new_param in zip(model.parameters(), params):
            param.data = new_param

def evaluate_model(model, test_dataset, batch_size=8):
    # テストデータセットからデータローダーを作成
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            output = model(inputs)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total
    return accuracy

def train(num_epochs, E):

    p = sum(i.numel() for i in agents_models[0].parameters())
    m = {i: torch.zeros(p) for i in range(N)}
    v = {i: torch.zeros(p) for i in range(N)}
    v_hat = {i: torch.zeros(p) for i in range(N)}
    x = {i: torch.zeros(p) for i in range(N)}
    y = {i: torch.zeros(p) for i in range(N)}
    x_half = {i: torch.zeros(p) for i in range(N)}
    y_half = {i: torch.zeros(p) for i in range(N)}
    #x_tilde = {i: torch.zeros(N, p) for i in range(N)}
    #y_tilde = {i: torch.zeros(N, p) for i in range(N)}
    x_tilde = {i: torch.zeros(p) for i in range(N)}
    y_tilde = {i: torch.zeros(p) for i in range(N)}
    v_hat_before =  {i: torch.zeros(p) for i in range(N)}
    u = {i: torch.zeros(p) for i in range(N)}
    beta1, beta2, beta3 = 0.9, 0.99, 0.7
    lamb = 0.97
    alpha = 0.005
    epsilon = 1e-8
    batch_size = 16

    trigger = [ 0 for i in range(N) ]
    W = [[1/N for i in range(N)] for j in range(N)]
    agents_loss_history = {i: [] for i in range(N)}
    trigger_history = [[0 for i in range(num_epochs)] for j in range(N)]
    accuracy_list = [[0 for i in range(num_epochs)] for j in range(N)]
    for epoch in range(num_epochs):
        
        agents_gradients = {i: [] for i in range(N)}  # エージェントごとの勾配を保存する辞書
        agents_losses = {i: [] for i in range(N)} 
        learning_rate = alpha / (epoch + 1 )
        epoch_losses = {i: 0.0 for i in range(N)}
        epoch_counts = {i: 0 for i in range(N)}
        
        for i in range(N):
            model = agents_models[i]
            train_dataset = train_agents_datasets[i]
            
            # 指定された範囲からランダムにインデックスを選択
            start_idx = (batch_size* 2 * epoch) % len(train_dataset)
            end_idx = (batch_size* 2 * (epoch + 1)) % len(train_dataset)
            if start_idx >= end_idx:
                mnp = start_idx
                start_idx = end_idx
                end_idx = mnp
            selected_indices = np.random.choice(range(start_idx, end_idx), size= 2*batch_size, replace=False)
            
            # 選択されたインデックスに基づいてデータセットのサブセットを作成
            subset = Subset(train_dataset, selected_indices)
            
            # サブセットからデータローダーを作成（バッチサイズは1に設定して、個々のサンプルを取得）
            dataloader = DataLoader(subset, batch_size=batch_size, shuffle=False)
            batch_losses = []  # バッチごとの損失を格納するリスト
            
            # データローダーからバッチを作成
            inputs_batch, labels_batch = [], []
            
            for inputs, labels in dataloader:
                model.zero_grad()
                output = model(inputs)
                loss = loss_function(output, labels)
                loss.backward()
                batch_losses.append(loss.item())

                gradients = torch.cat([param.grad.view(-1) for param in model.parameters() if param.grad is not None], 0)
                agents_gradients[i].append(gradients)
                epoch_losses[i] += loss.item()
                epoch_counts[i] += 1

            # エージェントの勾配リストを平均化
            agents_gradients[i] = torch.mean(torch.stack(agents_gradients[i]), dim=0)
            agents_losses[i] = np.mean(batch_losses)
        
        for i in range(N):
            if epoch_counts[i] > 0:
                average_loss = epoch_losses[i] / epoch_counts[i]
                agents_loss_history[i].append(average_loss)
        
        for i in range(N):
            m[i] = beta1* lamb** (epoch) * m[i] + (1- beta1* lamb**(epoch))* agents_gradients[i]
            v[i] = beta2 * v[i] + (1- beta2)* agents_gradients[i] * agents_gradients[i]
            v_hat_before[i] = v_hat[i]
            v_hat[i] = beta3 * v_hat[i] + (1 - beta3) * torch.max(v_hat_before[i], v[i])
            x_half[i] = x[i]
            y_half[i] = y_tilde[i]
            for j in range(N):
                if i != j:  
                    x_half[i] += W[i][j] * (x_tilde[j] - x_tilde[i])
                    y_half[i] += W[i][j] * (y_tilde[j] - y_tilde[i])
            
            u[i] = y_half[i] + epsilon
            u[i] = 1 / (torch.sqrt(u[i]))
            x[i] = x_half[i] - alpha * u[i] * m[i]
            y[i] = y_half[i] + v_hat[i] - v_hat_before[i]

            new_param = unflatten_parameters(x[i], param_shapes, param_elements)
            assign_params_to_model(agents_models[i], new_param)

            for i in range(N):
                if (torch.norm(x[i] - x_tilde[i]) > E*(1/((epoch+1)*(1/2)))) or  (torch.norm(y[i] - y_tilde[i]) > E*(1/((epoch+1)*(1/2)))):
                    trigger[i] += 1
                    x_tilde[i] = x[i]
                    y_tilde[i] = y[i]
                else:
                    x_tilde[i] = x_tilde[i]
                    y_tilde[i] = y_tilde[i]
                trigger_history[i][epoch] = trigger[i]
                
            for i in range(N):
                model = agents_models[i]
                test_dataset = test_agents_datasets[i]
                accuracy = evaluate_model(model, test_dataset)
                accuracy_list[i][epoch] = accuracy
            
    return  trigger_history,  accuracy_list